Make that we are running the correct version of TensorFlow first

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-rc0'

In [2]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Build a lines segmentation model based on unet architecture 

Here is [the paper](https://arxiv.org/pdf/1505.04597.pdf)

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')

from pathlib import Path
Path.ls = lambda x: list(x.iterdir())  # Source: https://github.com/fastai/fastai/blob/master/fastai/core.py#L236

from tqdm import tqdm

from PIL import Image

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical

In [4]:
from recognizer.datasets import Dataset
from recognizer.networks import unet
from recognizer.networks import NetworkInput

In [5]:
lines_detector_path = Dataset.processed_data_path()/"lines_detector"
lines_detector_images_path = lines_detector_path/"images"
lines_detector_masks_path = lines_detector_path/"masks"

Hyperparameters

In [6]:
batch_size = 16
epochs = 5

In [7]:
model_save_path = Path("../recognizer/weights/lines_detector.h5")

## Load dataset

In [8]:
x = []
y = []

sorted_image_names = sorted(lines_detector_images_path.glob("*.jpg"))
sorted_mask_names = sorted(lines_detector_masks_path.glob("*.png"))
for image_file_name, mask_file_name in tqdm(zip(sorted_image_names, sorted_mask_names)):
    image = img_to_array(Image.open(image_file_name)) / 255
    mask = img_to_array(Image.open(mask_file_name), dtype=np.uint8) // 127
    
    x.append(image)
    y.append(mask)

x = np.array(x)
y = np.array(y)

# len(x), x[0].shape
x.shape

1539it [00:03, 413.33it/s]


(1539, 256, 256, 1)

In [9]:
x[0].dtype, y[0].dtype

(dtype('float32'), dtype('uint8'))

## Augment the data

## Build the model

In [10]:
np.unique(y)

array([0, 1, 2], dtype=uint8)

In [11]:
input_shape = x[0].shape
number_of_classes = 3

y = to_categorical(y, number_of_classes)

In [12]:
model = unet(NetworkInput(input_shape=input_shape, number_of_classes=number_of_classes))

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 9248        dropout[0][0]                    
______________________________________________________________________________________________

## Train the model

Check progress in TensorBoard by running tensorboard --logdir=logs/lines_detector_model in terminal

In [13]:
# Try removing the old logs first
!rm -r ../logs/lines_detector_model

callbacks = [
    keras.callbacks.TensorBoard(log_dir='../logs/lines_detector_model')
]

In [14]:
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    x,
    y,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=.2,
    shuffle=True
)
history.history.keys()

Train on 1231 samples, validate on 308 samples
Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
  64/1231 [>.............................] - ETA: 11:13 - loss: 2.2344 - accuracy: 0.4839

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.show()

# Save the model

In [ ]:
model.save(model_save_path)

## Restore the model

In [ ]:
# model = keras.models.load_model(model_save_path)

# model.trainable = False
# model.compile()

## Predict on one image

In [ ]:
image_array = x[0]
test_batch = image_array.reshape((1,) + image_array.shape)

preds = model.predict(test_batch)
preds.shape

In [ ]:
array_to_img(preds[0])